In [24]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import sqlite3

from plotnine import *
from mizani.formatters import comma_format, percent_format
from datetime import datetime

import os
# from dotenv import load_dotenv
# load_dotenv()
import pandas_datareader as pdr

In [30]:
import sqlite3

In [31]:
fintech_research_sql = sqlite3.connect(database="../data/fintech_research.sqlite")

In [3]:
os.environ["WRDS_USER"] = ""
os.environ["WRDS_PASSWORD"] = ""

In [4]:
wrds_user = os.getenv("WRDS_USER")
wrds_password = os.getenv("WRDS_PASSWORD")

In [5]:
# print(os.environ.keys())

In [6]:
print(os.getenv('WRDS_USER'))

sjsj0101


In [7]:
## establish wrds connection

con = (
  "postgresql+psycopg2://"
 f"{os.getenv('WRDS_USER')}:{os.getenv('WRDS_PASSWORD')}"
  "@wrds-pgdata.wharton.upenn.edu:9737/wrds"
)

wrds = create_engine(con, pool_pre_ping=True)

In [8]:
## data range

start_date = "01/01/1970"
end_date = "12/31/2022"

In [9]:
## sql monthly data
crsp_monthly_query = (
  "SELECT msf.permno, msf.mthcaldt AS date, "
         "date_trunc('month', msf.mthcaldt)::date AS month, "
         "msf.mthret AS ret, msf.shrout, msf.mthprc AS altprc, "
         "ssih.primaryexch, ssih.siccd "
    "FROM crsp.msf_v2 AS msf "
    "INNER JOIN crsp.stksecurityinfohist AS ssih "
    "ON msf.permno = ssih.permno AND "
       "ssih.secinfostartdt <= msf.mthcaldt AND "
       "msf.mthcaldt <= ssih.secinfoenddt "
   f"WHERE msf.mthcaldt BETWEEN '{start_date}' AND '{end_date}' "
          "AND ssih.sharetype = 'NS' "
          "AND ssih.securitytype = 'EQTY' "  
          "AND ssih.securitysubtype = 'COM' " 
          "AND ssih.usincflg = 'Y' " 
          "AND ssih.issuertype in ('ACOR', 'CORP') " 
          "AND ssih.primaryexch in ('N', 'A', 'Q') "
          "AND ssih.conditionaltype in ('RW', 'NW') "
          "AND ssih.tradingstatusflg = 'A'"
)

crsp_monthly = (pd.read_sql_query(
    sql=crsp_monthly_query,
    con=wrds,
    dtype={"permno": int, "siccd": int},
    parse_dates={"date", "month"})
  .assign(shrout=lambda x: x["shrout"]*1000)
)

In [10]:
compustat_query = (
  "SELECT gvkey, datadate, seq, ceq, at, lt, txditc, txdb, itcb,  pstkrv, "
         "pstkl, pstk, capx, oancf, sale, cogs, xint, xsga "
    "FROM comp.funda "
    "WHERE indfmt = 'INDL' "
          "AND datafmt = 'STD' "
          "AND consol = 'C' "
         f"AND datadate BETWEEN '{start_date}' AND '{end_date}'"
)

compustat = pd.read_sql_query(
  sql=compustat_query,
  con=wrds,
  dtype={"gvkey": str},
  parse_dates={"datadate"}
)

In [13]:
crsp_monthly.to_csv('crsp_monthly_sample.csv',index=False)
crsp_monthly

,permno,date,month,ret,shrout,altprc,primaryexch,siccd
0,10000,1986-01-31,1986-01-01,0.707317,3.680000e+06,4.375000,Q,3990
1,10000,1986-02-28,1986-02-01,-0.257143,3.680000e+06,3.250000,Q,3990
2,10000,1986-03-31,1986-03-01,0.365385,3.680000e+06,4.437500,Q,3990
3,10000,1986-04-30,1986-04-01,-0.098592,3.793000e+06,4.000000,Q,3990
4,10000,1986-05-30,1986-05-01,-0.222656,3.793000e+06,3.109375,Q,3990
...,...,...,...,...,...,...,...,...
3150878,93436,2022-08-31,2022-08-01,-0.072489,3.133470e+09,275.610000,Q,9999
3150879,93436,2022-09-30,2022-09-01,-0.037589,3.158000e+09,265.250000,Q,9999
3150880,93436,2022-10-31,2022-10-01,-0.142168,3.157752e+09,227.540000,Q,9999
3150881,93436,2022-11-30,2022-11-01,-0.144326,3.157752e+09,194.700000,Q,9999


In [14]:
compustat.to_csv('compustat_monthly_sample.csv',index=False)
compustat

,gvkey,datadate,seq,ceq,at,lt,txditc,txdb,itcb,pstkrv,pstkl,pstk,capx,oancf,sale,cogs,xint,xsga
0,001000,1970-12-31,10.544,10.544,33.450,22.906,0.000,0.000,0.0,0.000,0.000,0.000,2.767,NaN,45.335,30.529,0.850,9.420
1,001000,1971-12-31,8.382,8.381,29.330,20.948,0.000,0.000,0.0,0.000,0.000,0.000,1.771,NaN,47.033,33.973,1.117,10.548
2,001000,1972-12-31,7.021,7.021,19.907,12.886,0.288,0.288,0.0,0.000,0.000,0.000,1.254,NaN,34.362,22.702,0.784,7.551
3,001000,1973-12-31,8.567,8.567,21.771,13.204,0.231,0.231,0.0,0.000,0.000,0.000,1.633,NaN,37.750,24.704,0.705,8.532
4,001000,1974-12-31,10.257,9.843,25.638,15.381,0.091,0.091,0.0,2.069,0.414,0.414,1.313,NaN,50.325,36.646,0.817,8.859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520104,353742,2022-11-30,11.413,11.413,15.488,4.075,0.000,0.000,0.0,0.000,0.000,0.000,3.385,-0.583,0.000,0.000,0.001,0.754
520105,353945,2022-12-31,283.784,283.784,981.551,685.045,32.811,32.811,0.0,0.000,0.000,0.000,34.425,133.388,780.538,404.969,0.555,200.689
520106,355398,2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
520107,355398,2022-12-31,239.806,239.806,576.112,334.075,3.611,3.611,0.0,0.000,0.000,0.000,60.471,68.321,558.225,418.322,12.493,60.286


In [35]:
## get fama french data, also available in french website

factors_ff3_monthly_raw = pdr.DataReader(
  name="F-F_Research_Data_Factors",
  data_source="famafrench", 
  start=start_date, 
  end=end_date)[0]

factors_ff3_monthly = (factors_ff3_monthly_raw
  .divide(100)
  .reset_index(names="month")
  .assign(month=lambda x: pd.to_datetime(x["month"].astype(str)))
  .rename(str.lower, axis="columns")
  .rename(columns={"mkt-rf": "mkt_excess"})
)

# factors_ff3_monthly.to_sql(name="factors_ff3_monthly", 
#           con=fintech_research_sql, 
#           if_exists="replace",
#           index=False)


factors_ff3_monthly

/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_34017/4081927975.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_34017/4081927975.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.


,month,mkt_excess,smb,hml,rf
0,1970-01-01,-0.0810,0.0293,0.0313,0.0060
1,1970-02-01,0.0513,-0.0258,0.0393,0.0062
2,1970-03-01,-0.0106,-0.0232,0.0399,0.0057
3,1970-04-01,-0.1100,-0.0615,0.0618,0.0050
4,1970-05-01,-0.0692,-0.0459,0.0333,0.0053
...,...,...,...,...,...
631,2022-08-01,-0.0377,0.0140,0.0029,0.0019
632,2022-09-01,-0.0935,-0.0081,0.0005,0.0019
633,2022-10-01,0.0783,0.0006,0.0801,0.0023
634,2022-11-01,0.0460,-0.0352,0.0138,0.0029


In [38]:
factors_ff3_daily_raw = pdr.DataReader(
  name="F-F_Research_Data_Factors_daily",
  data_source="famafrench", 
  start=start_date, 
  end=end_date)[0]

factors_ff3_daily = (factors_ff3_daily_raw
  .divide(100)
  .reset_index(names="date")
  .rename(str.lower, axis="columns")
  .rename(columns={"mkt-rf": "mkt_excess"})
)

factors_ff3_daily

/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_34017/921214661.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.


,date,mkt_excess,smb,hml,rf
0,1970-01-02,0.0118,0.0129,0.0101,0.00029
1,1970-01-05,0.0059,0.0067,0.0072,0.00029
2,1970-01-06,-0.0074,0.0010,0.0021,0.00029
3,1970-01-07,-0.0015,0.0040,-0.0033,0.00029
4,1970-01-08,0.0004,0.0018,-0.0017,0.00029
...,...,...,...,...,...
13364,2022-12-23,0.0051,-0.0061,0.0116,0.00016
13365,2022-12-27,-0.0051,-0.0074,0.0142,0.00016
13366,2022-12-28,-0.0123,-0.0024,-0.0029,0.00016
13367,2022-12-29,0.0187,0.0127,-0.0107,0.00016


In [27]:
factors_ff5_monthly_raw = pdr.DataReader(
  name="F-F_Research_Data_5_Factors_2x3",
  data_source="famafrench", 
  start=start_date, 
  end=end_date)[0]

factors_ff5_monthly = (factors_ff5_monthly_raw
  .divide(100)
  .reset_index(names="month")
  .assign(month=lambda x: pd.to_datetime(x["month"].astype(str)))
  .rename(str.lower, axis="columns")
  .rename(columns={"mkt-rf": "mkt_excess"})
)

factors_ff5_monthly

/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_34017/2420179673.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_34017/2420179673.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.


,month,mkt_excess,smb,hml,rmw,cma,rf
0,1970-01-01,-0.0810,0.0312,0.0313,-0.0172,0.0384,0.0060
1,1970-02-01,0.0513,-0.0276,0.0393,-0.0229,0.0276,0.0062
2,1970-03-01,-0.0106,-0.0241,0.0399,-0.0100,0.0429,0.0057
3,1970-04-01,-0.1100,-0.0640,0.0618,-0.0064,0.0621,0.0050
4,1970-05-01,-0.0692,-0.0448,0.0333,-0.0121,0.0390,0.0053
...,...,...,...,...,...,...,...
631,2022-08-01,-0.0377,0.0152,0.0029,-0.0475,0.0129,0.0019
632,2022-09-01,-0.0935,-0.0105,0.0005,-0.0151,-0.0080,0.0019
633,2022-10-01,0.0783,0.0189,0.0801,0.0334,0.0664,0.0023
634,2022-11-01,0.0460,-0.0274,0.0138,0.0638,0.0318,0.0029


In [28]:
industries_ff_monthly_raw = pdr.DataReader(
  name="10_Industry_Portfolios",
  data_source="famafrench", 
  start=start_date, 
  end=end_date)[0]

industries_ff_monthly = (industries_ff_monthly_raw
  .divide(100)
  .reset_index(names="month")
  .assign(month=lambda x: pd.to_datetime(x["month"].astype(str)))
  .rename(str.lower, axis="columns")
)
industries_ff_monthly

/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_34017/1024705127.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_34017/1024705127.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_34017/1024705127.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
/var/folders/cg/mzzfcskd2g16hq9zj4rdllw00000gn/T/ipykernel_34017/1024705127.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed

,month,nodur,durbl,manuf,enrgy,hitec,telcm,shops,hlth,utils,other
0,1970-01-01,-0.0424,-0.0825,-0.0730,-0.1145,-0.0906,-0.0257,-0.0588,-0.0712,-0.0423,-0.0883
1,1970-02-01,0.0468,0.0557,0.0514,0.0538,0.0294,0.0796,0.0523,0.0560,0.0836,0.0950
2,1970-03-01,-0.0078,0.0487,-0.0029,-0.0045,-0.0258,0.0229,-0.0169,-0.0508,0.0291,-0.0190
3,1970-04-01,-0.1012,-0.0886,-0.0897,-0.0917,-0.1434,-0.0879,-0.1108,-0.1011,-0.0805,-0.1368
4,1970-05-01,-0.0745,-0.0645,-0.0751,0.0261,-0.0955,-0.0375,-0.0945,-0.0778,-0.0536,-0.0825
...,...,...,...,...,...,...,...,...,...,...,...
631,2022-08-01,-0.0182,-0.0581,-0.0348,0.0332,-0.0525,-0.0286,-0.0314,-0.0503,0.0015,-0.0262
632,2022-09-01,-0.0835,-0.0655,-0.1154,-0.0914,-0.1142,-0.1385,-0.0763,-0.0191,-0.1162,-0.0805
633,2022-10-01,0.0998,-0.0661,0.1221,0.2361,0.0502,0.1058,0.0425,0.0880,0.0351,0.1166
634,2022-11-01,0.0530,-0.0810,0.0903,0.0099,0.0528,0.0215,0.0385,0.0535,0.0678,0.0599


In [29]:
## q factors

factors_q_monthly_link = (
  "https://global-q.org/uploads/1/2/2/6/122679606/"
  "q5_factors_monthly_2022.csv"
)

factors_q_monthly = (pd.read_csv(factors_q_monthly_link)
  .assign(
    month=lambda x: (
      pd.to_datetime(x["year"].astype(str) + "-" +
        x["month"].astype(str) + "-01"))
  )
  .drop(columns=["R_F", "R_MKT", "year"])
  .rename(columns=lambda x: x.replace("R_", "").lower())
  .query(f"month >= '{start_date}' and month <= '{end_date}'")
  .assign(
    **{col: lambda x: x[col]/100 for col in ["me", "ia", "roe", "eg"]}
  )
)

factors_q_monthly

,month,me,ia,roe,eg
36,1970-01-01,0.032798,0.032798,0.032798,0.032798
37,1970-02-01,0.001127,0.001127,0.001127,0.001127
38,1970-03-01,0.029901,0.029901,0.029901,0.029901
39,1970-04-01,0.057065,0.057065,0.057065,0.057065
40,1970-05-01,0.036207,0.036207,0.036207,0.036207
...,...,...,...,...,...
667,2022-08-01,-0.035470,-0.035470,-0.035470,-0.035470
668,2022-09-01,0.017172,0.017172,0.017172,0.017172
669,2022-10-01,-0.009359,-0.009359,-0.009359,-0.009359
670,2022-11-01,0.012883,0.012883,0.012883,0.012883


In [40]:
## macro

sheet_id = "1g4LOaRj4TvwJr9RIaA_nwrXXWTOy46bP"
sheet_name = "macro_predictors.xlsx"
macro_predictors_link = (
  f"https://docs.google.com/spreadsheets/d/{sheet_id}" 
  f"/gviz/tq?tqx=out:csv&sheet={sheet_name}"
)

macro_predictors = (
  pd.read_csv(macro_predictors_link, thousands=",")
  .assign(
    month=lambda x: pd.to_datetime(x["yyyymm"], format="%Y%m"),
    dp=lambda x: np.log(x["D12"])-np.log(x["Index"]),
    dy=lambda x: np.log(x["D12"])-np.log(x["Index"].shift(1)),
    ep=lambda x: np.log(x["E12"])-np.log(x["Index"]),
    de=lambda x: np.log(x["D12"])-np.log(x["E12"]),
    tms=lambda x: x["lty"]-x["tbl"],
    dfy=lambda x: x["BAA"]-x["AAA"]
  )
  .rename(columns={"b/m": "bm"})
  .get(["month", "dp", "dy", "ep", "de", "svar", "bm", 
        "ntis", "tbl", "lty", "ltr", "tms", "dfy", "infl"])
  .query("month >= @start_date and month <= @end_date")
  .dropna()
)
macro_predictors

,month,dp,dy,ep,de,svar,bm,ntis,tbl,lty,ltr,tms,dfy,infl
1188,1970-01-01,-3.291271,-3.370825,-2.697171,-0.594100,0.001067,0.700347,0.034473,0.0787,0.0693,-0.0021,-0.0094,0.0095,0.002653
1189,1970-02-01,-3.341549,-3.290196,-2.757287,-0.584261,0.001059,0.670148,0.035855,0.0713,0.0651,0.0587,-0.0062,0.0085,0.005291
1190,1970-03-01,-3.341958,-3.340507,-2.767581,-0.574378,0.000964,0.690327,0.035842,0.0663,0.0661,-0.0068,-0.0002,0.0079,0.005263
1191,1970-04-01,-3.246076,-3.340918,-2.679279,-0.566797,0.001615,0.736751,0.035712,0.0651,0.0699,-0.0413,0.0048,0.0087,0.007853
1192,1970-05-01,-3.182101,-3.245005,-2.622940,-0.559161,0.008152,0.774228,0.038661,0.0684,0.0743,-0.0468,0.0059,0.0087,0.002597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1819,2022-08-01,-4.110113,-4.153480,-3.042012,-1.068101,0.003349,0.227429,-0.009732,0.0263,0.0290,-0.0421,0.0027,0.0108,-0.000354
1820,2022-09-01,-4.005414,-4.103464,-2.953150,-1.052264,0.004934,0.249478,-0.011292,0.0313,0.0352,-0.0769,0.0039,0.0110,0.002151
1821,2022-10-01,-4.074095,-3.997260,-3.055849,-1.018245,0.006504,0.218935,-0.015252,0.0372,0.0398,-0.0139,0.0026,0.0116,0.004056
1822,2022-11-01,-4.118363,-4.066005,-3.134758,-0.983605,0.006343,0.207182,-0.017011,0.0415,0.0389,0.0268,-0.0026,0.0117,-0.001010


In [41]:
## cpi 
cpi_monthly = (pdr.DataReader(
    name="CPIAUCNS", 
    data_source="fred", 
    start=start_date, 
    end=end_date
  )
  .reset_index(names="month")
  .rename(columns={"CPIAUCNS": "cpi"})
  .assign(cpi=lambda x: x["cpi"]/x["cpi"].iloc[-1])
)

cpi_monthly

,month,cpi
0,1970-01-01,0.127360
1,1970-02-01,0.128034
2,1970-03-01,0.128708
3,1970-04-01,0.129718
4,1970-05-01,0.130055
...,...,...
631,2022-08-01,0.997891
632,2022-09-01,1.000037
633,2022-10-01,1.004094
634,2022-11-01,1.003080


In [17]:
## mktcap

crsp_monthly = (crsp_monthly
  .assign(mktcap=lambda x: x["shrout"]*x["altprc"]/1000000)
  .assign(mktcap=lambda x: x["mktcap"].replace(0, np.nan))
)

crsp_monthly

,permno,date,month,ret,shrout,altprc,primaryexch,siccd,mktcap
0,10000,1986-01-31,1986-01-01,0.707317,3.680000e+06,4.375000,Q,3990,16.100000
1,10000,1986-02-28,1986-02-01,-0.257143,3.680000e+06,3.250000,Q,3990,11.960000
2,10000,1986-03-31,1986-03-01,0.365385,3.680000e+06,4.437500,Q,3990,16.330000
3,10000,1986-04-30,1986-04-01,-0.098592,3.793000e+06,4.000000,Q,3990,15.172000
4,10000,1986-05-30,1986-05-01,-0.222656,3.793000e+06,3.109375,Q,3990,11.793859
...,...,...,...,...,...,...,...,...,...
3150878,93436,2022-08-31,2022-08-01,-0.072489,3.133470e+09,275.610000,Q,9999,863615.666700
3150879,93436,2022-09-30,2022-09-01,-0.037589,3.158000e+09,265.250000,Q,9999,837659.500000
3150880,93436,2022-10-31,2022-10-01,-0.142168,3.157752e+09,227.540000,Q,9999,718514.890080
3150881,93436,2022-11-30,2022-11-01,-0.144326,3.157752e+09,194.700000,Q,9999,614814.314400


In [18]:
## merge one month lagged mkt cap
mktcap_lag = (crsp_monthly
  .assign(
    month=lambda x: x["month"]+pd.DateOffset(months=1),
    mktcap_lag=lambda x: x["mktcap"]
  )
  .get(["permno", "month", "mktcap_lag"])
)

crsp_monthly = (crsp_monthly
  .merge(mktcap_lag, how="left", on=["permno", "month"])
)

crsp_monthly

,permno,date,month,ret,shrout,altprc,primaryexch,siccd,mktcap,mktcap_lag
0,10000,1986-01-31,1986-01-01,0.707317,3.680000e+06,4.375000,Q,3990,16.100000,NaN
1,10000,1986-02-28,1986-02-01,-0.257143,3.680000e+06,3.250000,Q,3990,11.960000,16.10000
2,10000,1986-03-31,1986-03-01,0.365385,3.680000e+06,4.437500,Q,3990,16.330000,11.96000
3,10000,1986-04-30,1986-04-01,-0.098592,3.793000e+06,4.000000,Q,3990,15.172000,16.33000
4,10000,1986-05-30,1986-05-01,-0.222656,3.793000e+06,3.109375,Q,3990,11.793859,15.17200
...,...,...,...,...,...,...,...,...,...,...
3150878,93436,2022-08-31,2022-08-01,-0.072489,3.133470e+09,275.610000,Q,9999,863615.666700,931110.61050
3150879,93436,2022-09-30,2022-09-01,-0.037589,3.158000e+09,265.250000,Q,9999,837659.500000,863615.66670
3150880,93436,2022-10-31,2022-10-01,-0.142168,3.157752e+09,227.540000,Q,9999,718514.890080,837659.50000
3150881,93436,2022-11-30,2022-11-01,-0.144326,3.157752e+09,194.700000,Q,9999,614814.314400,718514.89008


In [19]:
def assign_exchange(primaryexch):
    if primaryexch == "N":
        return "NYSE"
    elif primaryexch == "A":
        return "AMEX"
    elif primaryexch == "Q":
        return "NASDAQ"
    else:
        return "Other"

crsp_monthly["exchange"] = (crsp_monthly["primaryexch"]
  .apply(assign_exchange)
)

In [20]:
def assign_industry(siccd):
    if 1 <= siccd <= 999:
        return "Agriculture"
    elif 1000 <= siccd <= 1499:
        return "Mining"
    elif 1500 <= siccd <= 1799:
        return "Construction"
    elif 2000 <= siccd <= 3999:
        return "Manufacturing"
    elif 4000 <= siccd <= 4899:
        return "Transportation"
    elif 4900 <= siccd <= 4999:
        return "Utilities"
    elif 5000 <= siccd <= 5199:
        return "Wholesale"
    elif 5200 <= siccd <= 5999:
        return "Retail"
    elif 6000 <= siccd <= 6799:
        return "Finance"
    elif 7000 <= siccd <= 8999:
        return "Services"
    elif 9000 <= siccd <= 9999:
        return "Public"
    else:
        return "Missing"

crsp_monthly["industry"] = (crsp_monthly["siccd"]
  .apply(assign_industry)
)

In [42]:

crsp_monthly = (crsp_monthly
  .merge(factors_ff3_monthly, how="left", on="month")
  .assign(ret_excess=lambda x: x["ret"]-x["rf"])
  .assign(ret_excess=lambda x: x["ret_excess"].clip(lower=-1))
  .drop(columns=["rf"])
)

In [44]:
## drop na

crsp_monthly = (crsp_monthly
  .dropna(subset=["ret_excess", "mktcap", "mktcap_lag"])
)
crsp_monthly

,permno,date,month,ret,shrout,altprc,primaryexch,siccd,mktcap,mktcap_lag,exchange,industry,mkt_excess,smb,hml,ret_excess
1,10000,1986-02-28,1986-02-01,-0.257143,3.680000e+06,3.250000,Q,3990,11.960000,16.100000,NASDAQ,Manufacturing,0.0713,-0.0054,-0.0072,-0.262443
2,10000,1986-03-31,1986-03-01,0.365385,3.680000e+06,4.437500,Q,3990,16.330000,11.960000,NASDAQ,Manufacturing,0.0488,-0.0059,-0.0039,0.359385
3,10000,1986-04-30,1986-04-01,-0.098592,3.793000e+06,4.000000,Q,3990,15.172000,16.330000,NASDAQ,Manufacturing,-0.0131,0.0278,-0.0287,-0.103792
4,10000,1986-05-30,1986-05-01,-0.222656,3.793000e+06,3.109375,Q,3990,11.793859,15.172000,NASDAQ,Manufacturing,0.0462,-0.0135,-0.0021,-0.227556
5,10000,1986-06-30,1986-06-01,-0.005025,3.793000e+06,3.093750,Q,3990,11.734594,11.793859,NASDAQ,Manufacturing,0.0103,-0.0096,0.0128,-0.010225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3150878,93436,2022-08-31,2022-08-01,-0.072489,3.133470e+09,275.610000,Q,9999,863615.666700,931110.610500,NASDAQ,Public,-0.0377,0.0140,0.0029,-0.074389
3150879,93436,2022-09-30,2022-09-01,-0.037589,3.158000e+09,265.250000,Q,9999,837659.500000,863615.666700,NASDAQ,Public,-0.0935,-0.0081,0.0005,-0.039489
3150880,93436,2022-10-31,2022-10-01,-0.142168,3.157752e+09,227.540000,Q,9999,718514.890080,837659.500000,NASDAQ,Public,0.0783,0.0006,0.0801,-0.144468
3150881,93436,2022-11-30,2022-11-01,-0.144326,3.157752e+09,194.700000,Q,9999,614814.314400,718514.890080,NASDAQ,Public,0.0460,-0.0352,0.0138,-0.147226


In [46]:
## caculate be, op
compustat = (compustat
  .assign(
    be=lambda x: 
      (x["seq"].combine_first(x["ceq"]+x["pstk"])
       .combine_first(x["at"]-x["lt"])+
       x["txditc"].combine_first(x["txdb"]+x["itcb"]).fillna(0)-
       x["pstkrv"].combine_first(x["pstkl"])
       .combine_first(x["pstk"]).fillna(0))
  )
  .assign(
    be=lambda x: x["be"].apply(lambda y: np.nan if y <= 0 else y)
  )
  .assign(
    op=lambda x: 
      ((x["sale"]-x["cogs"].fillna(0)- 
        x["xsga"].fillna(0)-x["xint"].fillna(0))/x["be"])
  )
)

compustat

,gvkey,datadate,seq,ceq,at,lt,txditc,txdb,itcb,pstkrv,pstkl,pstk,capx,oancf,sale,cogs,xint,xsga,be,op
0,001000,1970-12-31,10.544,10.544,33.450,22.906,0.000,0.000,0.0,0.000,0.000,0.000,2.767,NaN,45.335,30.529,0.850,9.420,10.544,0.430197
1,001000,1971-12-31,8.382,8.381,29.330,20.948,0.000,0.000,0.0,0.000,0.000,0.000,1.771,NaN,47.033,33.973,1.117,10.548,8.382,0.166428
2,001000,1972-12-31,7.021,7.021,19.907,12.886,0.288,0.288,0.0,0.000,0.000,0.000,1.254,NaN,34.362,22.702,0.784,7.551,7.309,0.454919
3,001000,1973-12-31,8.567,8.567,21.771,13.204,0.231,0.231,0.0,0.000,0.000,0.000,1.633,NaN,37.750,24.704,0.705,8.532,8.798,0.432939
4,001000,1974-12-31,10.257,9.843,25.638,15.381,0.091,0.091,0.0,2.069,0.414,0.414,1.313,NaN,50.325,36.646,0.817,8.859,8.279,0.483513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520104,353742,2022-11-30,11.413,11.413,15.488,4.075,0.000,0.000,0.0,0.000,0.000,0.000,3.385,-0.583,0.000,0.000,0.001,0.754,11.413,-0.066153
520105,353945,2022-12-31,283.784,283.784,981.551,685.045,32.811,32.811,0.0,0.000,0.000,0.000,34.425,133.388,780.538,404.969,0.555,200.689,316.595,0.550625
520106,355398,2021-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
520107,355398,2022-12-31,239.806,239.806,576.112,334.075,3.611,3.611,0.0,0.000,0.000,0.000,60.471,68.321,558.225,418.322,12.493,60.286,243.417,0.275757


In [48]:
## compute inv

compustat_lag = (compustat
  .get(["gvkey", "year", "at"])
  .assign(year=lambda x: x["year"]+1)
  .rename(columns={"at": "at_lag"})
)

compustat = (compustat
  .merge(compustat_lag, how="left", on=["gvkey", "year"])
  .assign(inv=lambda x: x["at"]/x["at_lag"]-1)
  .assign(inv=lambda x: np.where(x["at_lag"] <= 0, np.nan, x["inv"]))
)

In [49]:
## keep last for duplicates
compustat = (compustat
  .assign(year=lambda x: pd.DatetimeIndex(x["datadate"]).year)
  .sort_values("datadate")
  .groupby(["gvkey", "year"])
  .tail(1)
  .reset_index()
)

In [51]:
## ccm link

ccm_linktable_query = (
  "SELECT lpermno AS permno, gvkey, linkdt, "
         "COALESCE(linkenddt, CURRENT_DATE) AS linkenddt "
    "FROM crsp.ccmxpf_linktable "
    "WHERE linktype IN ('LU', 'LC') "
          "AND linkprim IN ('P', 'C') "
          "AND usedflag = 1"
)

ccm_linktable = pd.read_sql_query(
  sql=ccm_linktable_query,
  con=wrds,
  dtype={"permno": int, "gvkey": str},
  parse_dates={"linkdt", "linkenddt"}
)

ccm_linktable

,permno,gvkey,linkdt,linkenddt
0,25881,001000,1970-11-13,1978-06-30
1,10015,001001,1983-09-20,1986-07-31
2,10023,001002,1972-12-14,1973-06-05
3,10031,001003,1983-12-07,1989-08-16
4,54594,001004,1972-04-24,2024-06-06
...,...,...,...,...
32490,23514,349994,2022-11-15,2024-06-06
32491,22205,350681,2021-10-22,2023-03-31
32492,16161,351038,2021-10-29,2024-06-06
32493,23773,352262,2023-03-17,2024-06-06


In [ ]:
ccm_links = (crsp_monthly
  .merge(ccm_linktable, how="inner", on="permno")
  .query("~gvkey.isnull() & (date >= linkdt) & (date <= linkenddt)")
  .get(["permno", "gvkey", "date"])
)

crsp_monthly = (crsp_monthly
  .merge(ccm_links, how="left", on=["permno", "date"])
)


In [55]:
crsp_monthly

,permno,date,month,ret,shrout,altprc,primaryexch,siccd,mktcap,mktcap_lag,exchange,industry,mkt_excess,smb,hml,ret_excess,gvkey
0,10000,1986-02-28,1986-02-01,-0.257143,3.680000e+06,3.250000,Q,3990,11.960000,16.100000,NASDAQ,Manufacturing,0.0713,-0.0054,-0.0072,-0.262443,013007
1,10000,1986-03-31,1986-03-01,0.365385,3.680000e+06,4.437500,Q,3990,16.330000,11.960000,NASDAQ,Manufacturing,0.0488,-0.0059,-0.0039,0.359385,013007
2,10000,1986-04-30,1986-04-01,-0.098592,3.793000e+06,4.000000,Q,3990,15.172000,16.330000,NASDAQ,Manufacturing,-0.0131,0.0278,-0.0287,-0.103792,013007
3,10000,1986-05-30,1986-05-01,-0.222656,3.793000e+06,3.109375,Q,3990,11.793859,15.172000,NASDAQ,Manufacturing,0.0462,-0.0135,-0.0021,-0.227556,013007
4,10000,1986-06-30,1986-06-01,-0.005025,3.793000e+06,3.093750,Q,3990,11.734594,11.793859,NASDAQ,Manufacturing,0.0103,-0.0096,0.0128,-0.010225,013007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3109996,93436,2022-08-31,2022-08-01,-0.072489,3.133470e+09,275.610000,Q,9999,863615.666700,931110.610500,NASDAQ,Public,-0.0377,0.0140,0.0029,-0.074389,184996
3109997,93436,2022-09-30,2022-09-01,-0.037589,3.158000e+09,265.250000,Q,9999,837659.500000,863615.666700,NASDAQ,Public,-0.0935,-0.0081,0.0005,-0.039489,184996
3109998,93436,2022-10-31,2022-10-01,-0.142168,3.157752e+09,227.540000,Q,9999,718514.890080,837659.500000,NASDAQ,Public,0.0783,0.0006,0.0801,-0.144468,184996
3109999,93436,2022-11-30,2022-11-01,-0.144326,3.157752e+09,194.700000,Q,9999,614814.314400,718514.890080,NASDAQ,Public,0.0460,-0.0352,0.0138,-0.147226,184996


In [56]:
data_dict = {
    "crsp_monthly": crsp_monthly,
    "compustat":compustat,
    "factors_ff3_monthly": factors_ff3_monthly,
    "factors_ff5_monthly": factors_ff5_monthly,
    "factors_ff3_daily": factors_ff3_daily,
    "industries_ff_monthly": industries_ff_monthly, 
    "factors_q_monthly": factors_q_monthly,
    "macro_predictors": macro_predictors,
    "cpi_monthly": cpi_monthly,
    
}

for key, value in data_dict.items():
    value.to_sql(name=key,
                 con=fintech_research_sql, 
                 if_exists="replace",
                 index=False)